In [46]:
import langchain
import transformers
from ctransformers import AutoModelForCausalLM
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer

In [47]:
# globals
model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf"
instruction_model_path = f"{model_path}/gguf-model-f16.bin"
question = "Which is the latest album released by Iron Maiden?"

In [48]:
# load model
model = AutoModelForCausalLM.from_pretrained(instruction_model_path, hf=True, model_type="llama",  gpu_layers=50)

In [49]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

In [50]:
# load text generation pipeline
generate_text = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    do_sample=False,
)
pipeline = HuggingFacePipeline(pipeline=generate_text)

In [51]:
# generate result without agent / wikipedia 
result = generate_text(question)[0]["generated_text"]
print(f"LLM result: {result}")

LLM result: Which is the latest album released by Iron Maiden?

Answer: The latest album released by Iron Maiden is "The Book of Souls" which was released in 2015.


In [52]:
# load agent with wikipedia
tools = load_tools(["wikipedia"], llm=pipeline)
agent = initialize_agent(
    tools,
    pipeline,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True)

In [53]:
result = agent(question)['output']
print(f"ReACT result: {result}")

ReACT result: The latest album released by Iron Maiden is "Senjutsu" (2020)


In [54]:
langchain.debug = True
result = agent(question)['output']
langchain.debug = False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Which is the latest album released by Iron Maiden?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Which is the latest album released by Iron Maiden?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here)

In [55]:
print(f"ReACT result: {result}")

ReACT result: The latest album released by Iron Maiden is "Senjutsu" (2020)
